In [1]:
import cv2
from pathlib import Path
import numpy as np

In [2]:
figuresPath = Path(str(Path.joinpath(Path('E:/datasets/NDI_images'), '20220725/20220725/Observed_Crop_200x200pix')))

In [3]:
file_names = list(figuresPath.glob('*.jpg'))

In [4]:
img_file = np.random.choice(file_names)

In [13]:
img = cv2.imread(str(img_file), cv2.IMREAD_UNCHANGED)

In [28]:
import collections

max_value_dict = collections.defaultdict(int)
min_value_dict = collections.defaultdict(int)
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    max_value_dict[np.max(img)] += 1
    min_value_dict[np.min(img)] += 1
print('max_value_dict', max_value_dict)
print('min_value_dict', min_value_dict)

max_value_dict defaultdict(<class 'int'>, {255: 30, 251: 1, 253: 2, 248: 3, 252: 3, 249: 2, 246: 1, 254: 3, 245: 2, 244: 1, 243: 1, 239: 1, 250: 2, 241: 1, 247: 1})
min_value_dict defaultdict(<class 'int'>, {0: 54})


In [29]:
normalized_pic_path = Path.joinpath(figuresPath, 'normalized_pics')
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    if np.max(img) != 255:
        img_norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(str(Path.joinpath(normalized_pic_path, file_stem)), img_norm)
    else:
        cv2.imwrite(str(Path.joinpath(normalized_pic_path, file_stem)), img)

In [30]:
file_names = list(normalized_pic_path.glob('*.jpg'))
max_value_dict = collections.defaultdict(int)
min_value_dict = collections.defaultdict(int)
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    max_value_dict[np.max(img)] += 1
    min_value_dict[np.min(img)] += 1
print('max_value_dict', max_value_dict)
print('min_value_dict', min_value_dict)

max_value_dict defaultdict(<class 'int'>, {255: 38, 254: 11, 253: 3, 252: 2})
min_value_dict defaultdict(<class 'int'>, {0: 54})


In [34]:
file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    cv2.imshow('window', np.hstack([img, threshold]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [35]:
def on_mouse_rect(event, x, y, flag, mstate):
    if event == cv2.EVENT_LBUTTONDOWN:
        mstate['selection'] = 'ongoing'
        mstate['xybegin'] = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        width = mstate['xyend'][0] - mstate['xybegin'][0]
        height = mstate['xyend'][1] - mstate['xybegin'][1]
        if width > 0 and height > 0:
            mstate['selection'] = 'valid'
        else:
            mstate['selection'] = 'invalid'
    elif event == cv2.EVENT_RBUTTONDOWN:
        mstate['selection'] = 'invalid'

    if mstate['selection'] == 'ongoing':
        mstate['xyend'] = (x, y)

In [39]:
mstate = {
    'selection': 'invalid',
    'xybegin': (-1, -1),
    'xyend': (-1, -1),
}
rec_record = []
for file_name in file_names:
    cv2.namedWindow('result')
    cv2.setMouseCallback('result', on_mouse_rect, mstate)
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    while True:
        xbegin, ybegin = mstate['xybegin']
        xend, yend = mstate['xyend']
        if mstate['selection'] == 'ongoing':
            cv2.rectangle(img, (xbegin, ybegin), (xend, yend), color=0, thickness=2)
        cv2.imshow('result', img)
        key = cv2.waitKey(30)
        if key == ord('q'):
            rec_record.append([xbegin, ybegin, xend, yend])
            break
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [42]:
average_rec = np.mean(rec_record, axis=0)
average_rec

array([ 88.1,  82.7, 118.4, 115.5])

In [73]:
file_names = list(normalized_pic_path.glob('*.jpg'))
average_rec = [83, 80, 117, 120]
xbegin, ybegin, xend, yend = average_rec
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    # cv2.rectangle(img, (xbegin, ybegin), (xend, yend), color=0, thickness=2)
    new_img = img.copy()
    new_img[ybegin: yend, xbegin: xend] = 0
    cv2.imshow('window', np.hstack([img, new_img]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [74]:
mask_pics_path = Path.joinpath(figuresPath, 'masked_pics')
file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    img[xbegin: xend, ybegin: yend] = 0
    cv2.imwrite(str(Path.joinpath(mask_pics_path, file_stem)), img)

In [5]:
mask_pics_path = Path.joinpath(figuresPath, 'masked_pics')
file_names = list(mask_pics_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 90, 255, cv2.THRESH_BINARY)
    cv2.imshow('window', np.hstack([img, threshold]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [83]:
# Useless

file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result = cv2.fastNlMeansDenoising(img, None, 10, 10)
    cv2.imshow('window', np.hstack([img, result]))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt

file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    fig, axes = plt.subplots(1, 2)
    axes[0].imshow(img)
    axes[1].plot(hist)
    axes[1].set_xlim([0, 256])
    plt.show()
    cv2.waitKey(0)

In [5]:
mask_pics_path = Path.joinpath(figuresPath, 'masked_pics')
file_names = list(mask_pics_path.glob('*.jpg'))
centrosymmetric_pics_path = Path.joinpath(figuresPath, 'centrosysmmetric_pics')
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    height, width = img.shape
    for i in range(height):
        for j in range(width):
            img[j, i] = max(img[j, i], img[width - j - 1, height - i - 1])
    cv2.imwrite(str(Path.joinpath(centrosymmetric_pics_path, file_stem)), img)

In [10]:
centrosymmetric_pics_path = Path.joinpath(figuresPath, 'centrosysmmetric_pics')
file_names = list(centrosymmetric_pics_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    cv2.imshow('window', np.hstack([img, threshold]))
    key = cv2.waitKey(0)
    if key == ord('q'):
        cv2.destroyAllWindows()
        break
    cv2.destroyAllWindows()

In [11]:
binaryzation_pics_path = Path.joinpath(figuresPath, 'binaryzation_pics')
file_names = list(centrosymmetric_pics_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    cv2.imwrite(str(Path.joinpath(binaryzation_pics_path, file_stem)), threshold)

In [12]:
stacked_pics_path = Path.joinpath(figuresPath, 'stacked_centrosymmetric_binaryzation_pics')
file_names = list(centrosymmetric_pics_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    img = np.hstack([img, threshold])
    cv2.imwrite(str(Path.joinpath(stacked_pics_path, file_stem)), img)

In [17]:
stacked_pics_path = Path.joinpath(figuresPath, 'stacked_original_binarization_pics')
file_names = list(binaryzation_pics_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    original_img = cv2.imread(str(Path.joinpath(figuresPath, file_stem)), cv2.IMREAD_UNCHANGED)
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    img = np.hstack([original_img, img])
    cv2.imwrite(str(Path.joinpath(stacked_pics_path, file_stem)), img)

In [43]:
import re
import collections

file_path = Path('E:/datasets/NDI_images/20220725/20220725')
text = open(str(Path.joinpath(file_path, '方位 (1).txt')), 'r', encoding='utf-8').readlines()
for i in range(len(text)):
    text[i] = re.search(r'\[.*?]', text[i]).group()
text_dict = collections.defaultdict(int)
for pattern in text:
    text_dict[pattern] += 1
sorted_dict = sorted(text_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_dict)
print(len(sorted_dict))

[('[4 -4 -1]', 10), ('[1 -10 8]', 9), ('[1 -1 -1]', 8), ('[1 -3 3]', 5), ('[7 -8 -35]', 3), ('[2 -2 -1]', 2), ('[1 -5 3]', 2), ('[-4 3 15]', 2), ('[3 2 -8]', 2), ('[-10 10 -47]', 2), ('[-6 5 -3]', 2), ('[3 7 -15]', 1), ('[-3 -2 7]', 1), ('[-4 3 12]', 1), ('[-6 14 -5]', 1), ('[52 -19 -10]', 1), ('[-1 3 -1]', 1), ('[4 -5 -8]', 1)]
18


In [5]:
target_images_path = Path('E:/datasets/NDI_images/20220725/20220725/Calculated_200x200')
grayscale_path = Path.joinpath(target_images_path, 'grayscale')
file_names = list(target_images_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    cv2.imwrite(str(Path.joinpath(grayscale_path, file_stem)), cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

In [6]:
file_names = list(grayscale_path.glob('*.jpg'))
img = cv2.imread(str(file_names[0]), cv2.IMREAD_UNCHANGED)
img.shape

(200, 200)

In [11]:
normalized_pic_path = Path.joinpath(figuresPath, 'normalized_pics')
file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    cv2.circle(img, center=(101, 100), radius=22, color=(255, 0, 0), thickness=1)
    cv2.imshow('test', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [17]:
circle_mask_pics_path = Path.joinpath(figuresPath, 'circle_masked_pics')
file_names = list(normalized_pic_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    height, width = img.shape
    for i in range(height):
        for j in range(width):
            if (i - 101) ** 2 + (j - 100) ** 2 <= 484:
                img[j, i] = 0
    cv2.imwrite(str(Path.joinpath(circle_mask_pics_path, file_stem)), img)

In [19]:
circle_mask_pics_path = Path.joinpath(figuresPath, 'circle_masked_pics')
file_names = list(circle_mask_pics_path.glob('*.jpg'))
circle_centrosymmetric_pics_path = Path.joinpath(figuresPath, 'circle_centrosysmmetric_pics')
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    height, width = img.shape
    for i in range(height):
        for j in range(width):
            img[j, i] = max(img[j, i], img[width - j - 1, height - i - 1])
    cv2.imwrite(str(Path.joinpath(circle_centrosymmetric_pics_path, file_stem)), img)

In [22]:
circle_binaryzation_pics_path = Path.joinpath(figuresPath, 'circle_binaryzation_pics')
file_names = list(circle_centrosymmetric_pics_path.glob('*.jpg'))
for file_name in file_names:
    file_stem = file_name.name
    img = cv2.imread(str(file_name), cv2.IMREAD_UNCHANGED)
    result, threshold = cv2.threshold(img, 110, 255, cv2.THRESH_BINARY)
    cv2.imwrite(str(Path.joinpath(circle_binaryzation_pics_path, file_stem)), threshold)